In [1]:
import pandas as pd
import numpy as np

In [2]:
import dynamic_portfolio.preprocess as prep
import dynamic_portfolio.utils as utils
import dynamic_portfolio.cross_validate as cv
import warnings

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

In [3]:
warnings.filterwarnings(action='ignore')

In [4]:
tickers = utils.return_tickers()

In [5]:
ticker_ind = 0
for ticker in tickers:
    ticker_ind += 1
    ready_to_train = []
    ready_to_train.append(prep.ready_to_train_df(ticker))
    print(f'ticker index = {ticker_ind}')
    print(f'done for ticker {ticker}')

ticker index = 1
done for ticker AAPL
ticker index = 2
done for ticker MSFT
ticker index = 3
done for ticker GOOG
ticker index = 4
done for ticker AMZN
ticker index = 5
done for ticker TSLA
ticker index = 6
done for ticker UNH
ticker index = 7
done for ticker XOM
ticker index = 8
done for ticker JNJ
ticker index = 9
done for ticker WMT
ticker index = 10
done for ticker NVDA
ticker index = 11
done for ticker JPM
ticker index = 12
done for ticker V
ticker index = 13
done for ticker CVX
ticker index = 14
done for ticker PG
ticker index = 15
done for ticker LLY
ticker index = 16
done for ticker MA
ticker index = 17
done for ticker HD
ticker index = 18
done for ticker META
ticker index = 19
done for ticker BAC
ticker index = 20
done for ticker ABBV
ticker index = 21
done for ticker PFE
ticker index = 22
done for ticker KO
ticker index = 23
done for ticker MRK
ticker index = 24
done for ticker PEP
ticker index = 25
done for ticker COST
ticker index = 26
done for ticker ORCL
ticker index = 27

ticker index = 214
done for ticker NEM
ticker index = 215
done for ticker DOW
ticker index = 216
done for ticker EA
ticker index = 217
done for ticker WBA
ticker index = 218
done for ticker DD
ticker index = 219
done for ticker ALL
ticker index = 220
done for ticker YUM
ticker index = 221
done for ticker AMP
ticker index = 222
done for ticker CMI
ticker index = 223
done for ticker ILMN
ticker index = 224
done for ticker TDG
ticker index = 225
done for ticker IDXX
ticker index = 226
done for ticker ED
ticker index = 227
done for ticker KR
ticker index = 228
done for ticker ABC
ticker index = 229
done for ticker DLTR
ticker index = 230
done for ticker RMD
ticker index = 231
done for ticker ALB
ticker index = 232
done for ticker HAL
ticker index = 233
done for ticker NDAQ
ticker index = 234
done for ticker LVS
ticker index = 235
done for ticker ODFL
ticker index = 236
done for ticker WELL
ticker index = 237
done for ticker AME
ticker index = 238
done for ticker CSGP
ticker index = 239
don

ticker index = 423
done for ticker HST
ticker index = 424
done for ticker UDR
ticker index = 425
done for ticker POOL
ticker index = 426
done for ticker RE
ticker index = 427
done for ticker PARA
ticker index = 428
done for ticker SNA
ticker index = 429
done for ticker CPT
ticker index = 430
done for ticker LW
ticker index = 431
done for ticker PKG
ticker index = 432
done for ticker CRL
ticker index = 433
done for ticker SWK
ticker index = 434
done for ticker BIO
ticker index = 435
done for ticker WDC
ticker index = 436
done for ticker CHRW
ticker index = 437
done for ticker STX
ticker index = 438
done for ticker MAS
ticker index = 439
done for ticker GL
ticker index = 440
done for ticker CE
ticker index = 441
done for ticker REG
ticker index = 442
done for ticker NI
ticker index = 443
done for ticker BXP
ticker index = 444
done for ticker HSIC
ticker index = 445
done for ticker CCL
ticker index = 446
done for ticker TFX
ticker index = 447
done for ticker CZR
ticker index = 448
done fo